Todo:

3. Prepare the whole pipeline
    1. Data augmentation
        - First take the unaugmented original dataset and proceed
        -  Augment the data with a predefined seed for each of the following techniques: rotation, flipping, contrast, brightness change, random erasing
    2. Choose hyperparameters
        - For the hyper-parameters related to the training process we chose batch size, learning rate, and number of epochs
        - For the hyper-parameters related to Regularization we decided to use L2 Regularization (Weight Decay) and Dropout Rate
    3. Train each of the prepared models on the augmented dataset for a chosen augmentation technique
    4. Test and collect data regarding models’ performance on the augmented dataset
    5. Repeat several times (>=3) from 3.
    6. Choose different values for hyperparameters and start from 3.
    7. Choose the next augmentation technique and start from 2.
    8. Repeat the process starting from 1. several times (>=3) with a different seed each time

In [11]:
import os
from enum import Enum

## Move 5400 images from each class from valid to train

In [19]:
rootdir = './Dataset/valid'

for subdir, dirs, files in os.walk(rootdir):
    if subdir != rootdir:
        for subsubdir, subdirs, files in os.walk(subdir):
            if len(files) < 5400:
                break;
            for i in range(5400):
                os.rename(os.path.join(os.path.join("./Dataset/valid",os.path.basename(subsubdir)),files[i]), os.path.join(os.path.join("./Dataset/train",os.path.basename(subsubdir)),files[i]))

## Pipeline

Helper functions:

In [12]:
class AugmentationTechnique(Enum):
    NoAugmentation = 0
    Rotation = 1
    Flipping = 2
    Contrast = 3
    Brightness = 4
    RandomErasing = 5

def augumentData(data, technique, seed):
    match technique:
        case AugmentationTechnique.Rotation:
            print("Hello")

    return data

def getAccuracy(y_result, y_test):
    correct_amount =0 
    for i, result in enumerate(y_result):
        if result == y_test[i]:
            correct_amount+=1
    return correct_amount/len(y_test)

Arrays containing different hyper-parameter values

TODO: Handle hyperparameters related to regularization

In [13]:
# training process
batchSizes =[]
learningRates = []
numberOfEpochs =[]

# regularization



# augmentation
augmentationTechniques =[AugmentationTechnique.NoAugmentation,AugmentationTechnique.Rotation,AugmentationTechnique.Flipping,AugmentationTechnique.Contrast,AugmentationTechnique.Brightness,AugmentationTechnique.RandomErasing]


REPETITIONS = 3
seeds = [123,42,56]

Main pipeline loop

TODO: repeat the experiments 3 times, different seed each time. I want to first test if it works once

In [ ]:
def performExperiment(model,X,y):
    results = [{'accuracy':0,'augmentation': AugmentationTechnique.Brightness,'batchSize':0,'learningRate':0,'numberOfEpochs':0}]
    for batchSize in batchSizes:
        for learningRate in learningRates:
            for epochNumber in numberOfEpochs:
                for augmentation in augmentationTechniques:
                    augX = augumentData(X,augmentation,seeds[0])
                    # train the model with hyperparameters

                    # fit the model

                    # get accuracy
                    getAccuracy()

                    # append to results
        

# UNUSED

In [ ]:


class HyperParameter(Enum):
    BatchSize=1
    LearningRate =2
    NumberOfEpochs =3

class HyperParameters:
    def __init__(self, batchSizes, learningRates, numberOfEpochs):
        self.batchSizes=batchSizes
        self.learningRates=learningRates
        self.numberOfEpochs=numberOfEpochs
        self.currentIndex =0
    
    def getNextHyperParameter(self):
        if self.currentIndex<len(self.batchSizes):
            return self.batchSizes[self.currentIndex], HyperParameter.BatchSize
        elif self.currentIndex<(len(self.batchSizes+self.learningRates)):
            return self.learningRates[self.currentIndex-len(self.batchSizes)], HyperParameter.LearningRate
        elif self.currentIndex<(len(self.batchSizes)+len(self.learningRates)+len(self.numberOfEpochs)):
            return self.num
        

